In [1]:
import os
import sqlite3
import pandas as pd
from datetime import datetime
from fuzzywuzzy import process

In [2]:
conn = sqlite3.connect("../data/db/NBA-L2M.db")
c = conn.cursor()

In [8]:
reports = pd.read_sql("SELECT * FROM reports", conn, parse_dates=['date'])
reports.replace({'away':{'PHO': 'PHX'}})
print reports.dtypes, '\r'
reports.head(2)

report                object
away                  object
home                  object
date          datetime64[ns]
season                object
away_score           float64
home_score           float64
winner                object
dtype: object 


,report,away,home,date,season,away_score,home_score,winner
0,L2M-DEN-MIN-11-03-16,DEN,MIN,2016-11-03,2016-17 NBA Regular Season,102.0,99.0,DEN
1,L2M-SAC-CHI-01-21-17,SAC,CHI,2017-01-21,2016-17 NBA Regular Season,99.0,102.0,CHI


In [9]:
calls = pd.read_sql("SELECT * FROM calls", conn)
print calls.dtypes, '\r'
calls.head(2)

report                object
period                object
time                  object
call                  object
committing            object
disadvantaged         object
decision              object
committing_team       object
disadvantaged_team    object
dtype: object 


,report,period,time,call,committing,disadvantaged,decision,committing_team,disadvantaged_team
0,L2M-DEN-MIN-11-03-16,Q4,,Foul: Shooting,Karl-Anthony Towns,Emmanuel Mudiay,CC,MIN,DEN
1,L2M-DEN-MIN-11-03-16,Q4,,Foul: Offensive,Karl-Anthony Towns,Danilo Gallinari,CNC,MIN,DEN


In [11]:
print len(reports)
print len(calls)

963
15625
